In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import io
import os
from pptx import Presentation
from pptx.util import Inches
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.dates as mdates

In [ ]:
# Path to csv data file
pathp = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Metals_Poros_PM2.5_PM0.1_forPython.csv'

# output_folder is the destination folder of all of the plots below
output_dir = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Python_Output'

In [ ]:
df = pd.read_csv(pathp)
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Filter out 'date', and any column containing 'sx3' or 's_x3'
cols_to_keep = [
    col for col in df.columns
    if col.lower() != 'date' and 'x3' not in col.lower()
]
df = df[cols_to_keep]

# Clean the column names by removing the specific unit strings
df.columns = [col.replace('_ug.m3', '').replace('_ng.m3', '') for col in df.columns]

# 2. Calculate Correlation Matrix and R-squared
# df.corr() gives 'r'. We square it to get R^2.
corr_matrix = df.corr(method='pearson')
r2_matrix = corr_matrix ** 2

# Calculate the number of common valid data points between each pair of columns
common_points = df.notna().astype(int).T.dot(df.notna().astype(int))

# 3. Create a Mask for the Upper Triangle AND cells with < 5 common points
mask = np.triu(np.ones_like(r2_matrix, dtype=bool))
mask = mask | (common_points < 5).values

# 4. Set up the Matplotlib Figure
f, ax = plt.subplots(figsize=(22, 22))

# 5. Define the Colormap
cmap = 'RdYlBu_r'

# 6. Draw the Heatmap
sns.heatmap(r2_matrix,
            mask=mask,
            cmap=cmap,
            vmin=0, vmax=1,
            center=0.5,
            square=True,
            linewidths=.5,
            annot=True,            # This turns on the numbers inside the boxes
            fmt=".3f",             # Formats the numbers to 3 decimal places (e.g., 0.001)
            annot_kws={"size": 8}, # Sets the font size for the numbers inside the boxes
            cbar_kws={"orientation": "horizontal", "shrink": 0.8, "pad": 0.05, "label": "$R^2$ Value"},
            xticklabels=True, yticklabels=True
           )

# 7. Customize Plot Labels and Title
# Increased fontsize to 12 since the names are shorter now
plt.xticks(rotation=90, fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.title('Correlation Heatmap of Metals and PM Species ($R^2$ Values)', fontsize=20, y=1.02)

ax.tick_params(axis='both', which='both', length=0)

# 8. Save the Plot
output_dir = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Python_Output'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'Correlation_Heatmap_Clean_Labels.png')

plt.tight_layout()
plt.savefig(output_path, dpi=300, bbox_inches='tight')

print(f"Heatmap saved successfully to: {output_path}")
plt.show()

## Chlorine vs Chloride

In [ ]:
# if you have run the Heatmap plot first you need to reload the file
df = pd.read_csv(pathp)
df['date'] = pd.to_datetime(df['date'])

# --- Column Name Variables ---
col_cl_pm25 = 'Cl_PM2.5_ug.m3'
col_cl_pm01 = 'Cl_pm0.1_ng.m3'
col_chl_pm1 = 'chl_pm1'
col_chloride_pm01 = 'Chloride_pm0.1_ng.m3'

# --- Clean Axis Labels Mapping ---
label_map = {
    col_cl_pm25: 'Chlorine PM2.5 Filters (WD-XRF)',
    col_cl_pm01: 'Chlorine PM0.1 Xact',
    col_chl_pm1: 'Chloride PM1 AMS',
    col_chloride_pm01: 'Chloride PM0.1 AMS'
}

# --- Font Sizes ---
title_size = 18
label_size = 16
tick_size = 14
line_thickness = 2.5

# 1. Define the pairs for the rows
#[(v1, v2, 'Title', dual-axis)]
plot_configs = [
    (col_cl_pm25, col_chl_pm1, 'PM2.5 and PM1 (Filters vs AMS)', False),
    (col_cl_pm25, col_cl_pm01, 'PM2.5 vs PM0.1 (Both XRF)', True),
    (col_chl_pm1, col_chloride_pm01, 'PM1 vs PM0.1 (Both AMS)', True),
    (col_cl_pm01, col_chloride_pm01, 'PM0.1 (Xact vs AMS)', False)
]

# 2. Create Figure: 4 rows, 2 columns. Increased height to 21.
fig, axes = plt.subplots(4, 2, figsize=(22, 21), gridspec_kw={'width_ratios': [3, 1]})

for i, (c1, c2, title, dual_axis) in enumerate(plot_configs):
    ax_time = axes[i, 0]
    ax_scat = axes[i, 1]

    # --- TIMESERIES X-AXIS TICKS ---
    ax_time.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=20))
    ax_time.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))

    # Increase tick label size for timeseries and rotate manually
    ax_time.tick_params(axis='both', which='major', labelsize=tick_size)
    plt.setp(ax_time.xaxis.get_majorticklabels(), rotation=30, ha='right')

    # --- LEFT PLOT: Timeseries ---
    if not dual_axis:
        ax_time.plot(df['date'], df[c1], color='black', label=label_map[c1], drawstyle='steps-post', linewidth=line_thickness)
        ax_time.plot(df['date'], df[c2], color='tab:red', label=label_map[c2], drawstyle='steps-post', linewidth=line_thickness)

        # Adjusting the Y-label dynamically: µg/m³ for the top plot, ng/m³ for the bottom plot
        y_label_text = 'Concentration (µg/m³)' if i == 0 else 'Concentration (ng/m³)'
        ax_time.set_ylabel(y_label_text, fontsize=label_size, fontweight='bold')
        ax_time.legend(loc='upper left', fontsize=tick_size)
    else:
        ax_time.plot(df['date'], df[c1], color='black', label=label_map[c1], drawstyle='steps-post', linewidth=line_thickness)
        ax_time.set_ylabel(label_map[c1], color='black', fontsize=label_size, fontweight='bold')
        ax_time.tick_params(axis='y', labelcolor='black', labelsize=tick_size)

        ax_twin = ax_time.twinx()
        color2 = 'tab:red' if i == 1 else 'tab:red'
        ax_twin.plot(df['date'], df[c2], color=color2, label=label_map[c2], drawstyle='steps-post', linestyle='--', linewidth=line_thickness)
        ax_twin.set_ylabel(label_map[c2], color=color2, fontsize=label_size, fontweight='bold')
        ax_twin.tick_params(axis='y', labelcolor=color2, labelsize=tick_size)

        # Ensure secondary Y-axis starts at zero
        ax_twin.set_ylim(bottom=0)

        lines1, labels1 = ax_time.get_legend_handles_labels()
        lines2, labels2 = ax_twin.get_legend_handles_labels()
        ax_time.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=tick_size)

    ax_time.set_title(title, fontsize=title_size)

    # Ensure primary Y-axis starts at zero
    ax_time.set_ylim(bottom=0)

    # Only set 'Date' label on the bottom timeseries (i == 3 for the 4th row)
    if i == 3:
        ax_time.set_xlabel('Date', fontsize=label_size)

    # --- RIGHT PLOT: Scatter with R2 & Best Fit Line ---
    scatter_data = df[[c1, c2]].dropna()
    if len(scatter_data) > 1:
        x_data = scatter_data[c1]
        y_data = scatter_data[c2]

        r_val = x_data.corr(y_data)
        r2_val = r_val**2

        # 1. Plot the scatter points
        ax_scat.scatter(x_data, y_data, alpha=0.6, color='tab:red')

        # 2. Calculate the best fit line (y = mx + b)
        m, b = np.polyfit(x_data, y_data, 1)
        x_line = np.array([0, x_data.max()])
        y_line = m * x_line + b
        ax_scat.plot(x_line, y_line, color='black', linestyle='--', linewidth=2.5)

        ax_scat.set_xlabel(label_map[c1], fontsize=label_size)
        ax_scat.set_ylabel(label_map[c2], fontsize=label_size)

        # Force scatter plot axes to zero
        ax_scat.set_xlim(left=0)
        ax_scat.set_ylim(bottom=0)

        # Format Text Box Content
        sign = '+' if b >= 0 else '-'
        eq_text = f'$y = {m:.3f}x {sign} {abs(b):.3f}$'
        r2_text = f'$R^2 = {r2_val:.3f}$'
        box_text = f'{r2_text}\n{eq_text}'

        # Display Text Box
        ax_scat.text(0.05, 0.95, box_text,
                     transform=ax_scat.transAxes, fontsize=label_size,
                     verticalalignment='top',
                     bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'))
    else:
        ax_scat.text(0.5, 0.5, 'Insufficient Data', transform=ax_scat.transAxes, ha='center', fontsize=label_size)

    ax_scat.set_title(f'{label_map[c1]} vs {label_map[c2]}', fontsize=title_size)
    ax_scat.tick_params(axis='both', which='major', labelsize=tick_size, labelbottom=True)

# 3. Format and Save
plt.tight_layout()

# output_dir = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Python_Output'
# os.makedirs(output_dir, exist_ok=True)
# output_path = os.path.join(output_dir, 'Chlorine_Chloride_4Panel.png')
# plt.savefig(output_path, dpi=300, bbox_inches='tight')
# print(f"Plot saved successfully to: {output_path}")

plt.show()

## Sulfur vs Sulfate

In [ ]:
# if you have run the Heatmap plot first and have not run the 'Chlorine vs Chloride' cell afterwards, you need to reload the file here
# df = pd.read_csv(pathp)
# df['date'] = pd.to_datetime(df['date'])

# --- Column Name Variables ---
col_s_pm25 = 'S_x3_PM2.5_ug.m3'
col_sulf_pm1 = 'sulf_pm1'
col_sx3_pm01 = 'Sx3_pm0.1_ng.m3'
col_sulf_pm01 = 'Sulfate_pm0.1_ng.m3'

# --- Clean Axis Labels Mapping ---
label_map = {
    col_s_pm25: 'Sulfur PM2.5 (x3) Filters',
    col_sulf_pm1: 'Sulfate PM1 AMS',
    col_sx3_pm01: 'Sulfur PM0.1 (x3) Xact',
    col_sulf_pm01: 'Sulfate PM0.1 AMS'
}

# --- Font Sizes ---
title_size = 18
label_size = 16
tick_size = 14
line_thickness = 2.5

# 1. Define the pairs for the rows
# [(v1, v2, 'Title', dual-axis)]
plot_configs = [
    (col_s_pm25, col_sulf_pm1, 'PM2.5 and PM1 (Filters vs AMS)', False),
    (col_s_pm25, col_sx3_pm01, 'PM2.5 vs PM0.1 (Both XRF)', True),
    (col_sulf_pm1, col_sulf_pm01, 'PM1 vs PM0.1 (Both AMS)', True),
    (col_sx3_pm01, col_sulf_pm01, 'PM0.1 (Xact vs AMS)', False)
]

# 2. Create Figure: 4 rows, 2 columns
fig, axes = plt.subplots(4, 2, figsize=(22, 21), gridspec_kw={'width_ratios': [3, 1]})

for i, (c1, c2, title, dual_axis) in enumerate(plot_configs):
    ax_time = axes[i, 0]
    ax_scat = axes[i, 1]

    # --- TIMESERIES X-AXIS TICKS ---
    ax_time.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=20))
    ax_time.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))

    ax_time.tick_params(axis='both', which='major', labelsize=tick_size)
    plt.setp(ax_time.xaxis.get_majorticklabels(), rotation=30, ha='right')

    # --- LEFT PLOT: Timeseries ---
    if not dual_axis:
        ax_time.plot(df['date'], df[c1], color='black', label=label_map[c1], linewidth=line_thickness)
        ax_time.plot(df['date'], df[c2], color='tab:red', label=label_map[c2], linewidth=line_thickness)

        # Adjusting the Y-label dynamically based on which row we are plotting
        y_label_text = 'Concentration (µg/m³)' if i == 0 else 'Concentration (ng/m³)'
        ax_time.set_ylabel(y_label_text, fontsize=label_size, fontweight='bold')
        ax_time.legend(loc='upper left', fontsize=tick_size)
    else:
        ax_time.plot(df['date'], df[c1], color='black', label=label_map[c1], linewidth=line_thickness)
        ax_time.set_ylabel(label_map[c1], color='black', fontsize=label_size, fontweight='bold')
        ax_time.tick_params(axis='y', labelcolor='black', labelsize=tick_size)

        ax_twin = ax_time.twinx()
        color2 = 'tab:red'
        ax_twin.plot(df['date'], df[c2], color=color2, label=label_map[c2], linestyle='--', linewidth=line_thickness)
        ax_twin.set_ylabel(label_map[c2], color=color2, fontsize=label_size, fontweight='bold')
        ax_twin.tick_params(axis='y', labelcolor=color2, labelsize=tick_size)

        ax_twin.set_ylim(bottom=0)

        lines1, labels1 = ax_time.get_legend_handles_labels()
        lines2, labels2 = ax_twin.get_legend_handles_labels()
        ax_time.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=tick_size)

    ax_time.set_title(title, fontsize=title_size)
    ax_time.set_ylim(bottom=0)

    if i == 3:
        ax_time.set_xlabel('Date', fontsize=label_size)

    # --- RIGHT PLOT: Scatter with R2 & Best Fit Line ---
    scatter_data = df[[c1, c2]].dropna()
    if len(scatter_data) > 1:
        x_data = scatter_data[c1]
        y_data = scatter_data[c2]

        r_val = x_data.corr(y_data)
        r2_val = r_val**2

        # Plot the scatter points
        ax_scat.scatter(x_data, y_data, alpha=0.6, color='tab:green')

        # Calculate the best fit line (y = mx + b)
        m, b = np.polyfit(x_data, y_data, 1)

        x_line = np.array([0, x_data.max()])
        y_line = m * x_line + b
        ax_scat.plot(x_line, y_line, color='tab:red', linestyle='--', linewidth=2.5)

        ax_scat.set_xlabel(label_map[c1], fontsize=label_size)
        ax_scat.set_ylabel(label_map[c2], fontsize=label_size)

        ax_scat.set_xlim(left=0)
        ax_scat.set_ylim(bottom=0)

        # --- Format Text Box Content ---
        sign = '+' if b >= 0 else '-'
        eq_text = f'$y = {m:.3f}x {sign} {abs(b):.3f}$'
        r2_text = f'$R^2 = {r2_val:.3f}$'
        box_text = f'{r2_text}\n{eq_text}'

        # Display Text Box
        ax_scat.text(0.05, 0.95, box_text,
                     transform=ax_scat.transAxes, fontsize=label_size,
                     verticalalignment='top',
                     bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'))
    else:
        ax_scat.text(0.5, 0.5, 'Insufficient Data', transform=ax_scat.transAxes, ha='center', fontsize=label_size)

    ax_scat.set_title(f'{label_map[c1]} vs {label_map[c2]}', fontsize=title_size)
    ax_scat.tick_params(axis='both', which='major', labelsize=tick_size, labelbottom=True)

# 3. Format and Save
plt.tight_layout()

# output_dir = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Python_Output'
# os.makedirs(output_dir, exist_ok=True)
# output_path = os.path.join(output_dir, 'Sulfur_4Panel_Timeseries_and_Scatter.png')
# plt.savefig(output_path, dpi=300, bbox_inches='tight')
# print(f"Plot saved successfully to: {output_path}")

plt.show()

## Calcium and Potassium

In [ ]:
# df = pd.read_csv(pathp)
# df['date'] = pd.to_datetime(df['date'])

# --- Column Name Variables (Double check these match your dataframe!) ---
col_ca_pm25 = 'Ca_PM2.5_ug.m3'
col_ca_pm01 = 'Ca_pm0.1_ng.m3'

col_k_pm25 = 'K_PM2.5_ug.m3'
col_k_pm01 = 'K_pm0.1_ng.m3'

col_zn_pm25 = 'Zn_PM2.5_ug.m3'
col_zn_pm01 = 'Zn_pm0.1_ng.m3'

col_fe_pm25 = 'Fe_PM2.5_ug.m3'
col_fe_pm01 = 'Fe_pm0.1_ng.m3'

# --- Clean Axis Labels Mapping ---
label_map = {
    col_ca_pm25: 'Calcium PM2.5 (µg/m³)',
    col_ca_pm01: 'Calcium PM0.1 (ng/m³)',
    col_k_pm25: 'Potassium PM2.5 (µg/m³)',
    col_k_pm01: 'Potassium PM0.1 (ng/m³)',
    col_zn_pm25: 'Zinc PM2.5 (µg/m³)',
    col_zn_pm01: 'Zinc PM0.1 (ng/m³)',
    col_fe_pm25: 'Iron PM2.5 (µg/m³)',
    col_fe_pm01: 'Iron PM0.1 (ng/m³)'
}

# --- Font Sizes ---
title_size = 18
label_size = 16
tick_size = 14
line_thickness = 2.5

# 1. Define the pairs for the rows
# All use dual_axis=True since we are comparing µg/m³ to ng/m³
plot_configs = [
    (col_ca_pm25, col_ca_pm01, 'Calcium: PM2.5 vs PM0.1', True),
    (col_k_pm25, col_k_pm01, 'Potassium: PM2.5 vs PM0.1', True),
    (col_zn_pm25, col_zn_pm01, 'Zinc: PM2.5 vs PM0.1', True),
    (col_fe_pm25, col_fe_pm01, 'Iron: PM2.5 vs PM0.1', True)
]

# 2. Create Figure: 4 rows, 2 columns. Adjusted height to 21.
fig, axes = plt.subplots(4, 2, figsize=(22, 21), gridspec_kw={'width_ratios': [3, 1]})

for i, (c1, c2, title, dual_axis) in enumerate(plot_configs):
    ax_time = axes[i, 0]
    ax_scat = axes[i, 1]

    # --- TIMESERIES X-AXIS TICKS ---
    ax_time.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=20))
    ax_time.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))

    ax_time.tick_params(axis='both', which='major', labelsize=tick_size)
    plt.setp(ax_time.xaxis.get_majorticklabels(), rotation=30, ha='right')

    # --- LEFT PLOT: Timeseries ---
    if not dual_axis:
        ax_time.plot(df['date'], df[c1], color='tab:blue', label=label_map[c1], drawstyle='steps-post', linewidth=line_thickness)
        ax_time.plot(df['date'], df[c2], color='tab:red', label=label_map[c2], drawstyle='steps-post', linewidth=line_thickness)
        ax_time.set_ylabel('Concentration', fontsize=label_size, fontweight='bold')
        ax_time.legend(loc='upper left', fontsize=tick_size)
    else:
        # Primary Axis (PM2.5)
        ax_time.plot(df['date'], df[c1], color='tab:blue', label=label_map[c1], drawstyle='steps-post', linewidth=line_thickness)
        ax_time.set_ylabel(label_map[c1], color='tab:blue', fontsize=label_size, fontweight='bold')
        ax_time.tick_params(axis='y', labelcolor='tab:blue', labelsize=tick_size)

        # Secondary Axis (PM0.1)
        ax_twin = ax_time.twinx()
        color2 = 'tab:red'
        ax_twin.plot(df['date'], df[c2], color=color2, label=label_map[c2], drawstyle='steps-post', linestyle='--', linewidth=line_thickness)
        ax_twin.set_ylabel(label_map[c2], color=color2, fontsize=label_size, fontweight='bold')
        ax_twin.tick_params(axis='y', labelcolor=color2, labelsize=tick_size)

        ax_twin.set_ylim(bottom=0)

        lines1, labels1 = ax_time.get_legend_handles_labels()
        lines2, labels2 = ax_twin.get_legend_handles_labels()
        ax_time.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=tick_size)

    ax_time.set_title(title, fontsize=title_size)
    ax_time.set_ylim(bottom=0)

    # Only set 'Date' label on the bottom timeseries (i == 3 for a 4-row plot)
    if i == 3:
        ax_time.set_xlabel('Date', fontsize=label_size)

    # --- RIGHT PLOT: Scatter with R2 & Best Fit Line ---
    scatter_data = df[[c1, c2]].dropna()
    if len(scatter_data) > 1:
        x_data = scatter_data[c1]
        y_data = scatter_data[c2]

        r_val = x_data.corr(y_data)
        r2_val = r_val**2

        # 1. Plot the scatter points
        ax_scat.scatter(x_data, y_data, alpha=0.6, color='tab:green')

        # 2. Calculate the best fit line (y = mx + b)
        m, b = np.polyfit(x_data, y_data, 1)
        x_line = np.array([0, x_data.max()])
        y_line = m * x_line + b
        ax_scat.plot(x_line, y_line, color='black', linestyle='--', linewidth=2.5)

        ax_scat.set_xlabel(label_map[c1], fontsize=label_size)
        ax_scat.set_ylabel(label_map[c2], fontsize=label_size)

        # Force scatter plot axes to zero
        ax_scat.set_xlim(left=0)
        ax_scat.set_ylim(bottom=0)

        # Format Text Box Content
        sign = '+' if b >= 0 else '-'
        eq_text = f'$y = {m:.3f}x {sign} {abs(b):.3f}$'
        r2_text = f'$R^2 = {r2_val:.3f}$'
        box_text = f'{r2_text}\n{eq_text}'

        # Display Text Box
        ax_scat.text(0.05, 0.95, box_text,
                     transform=ax_scat.transAxes, fontsize=label_size,
                     verticalalignment='top',
                     bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'))
    else:
        ax_scat.text(0.5, 0.5, 'Insufficient Data', transform=ax_scat.transAxes, ha='center', fontsize=label_size)

    ax_scat.set_title(f'{label_map[c1]} vs {label_map[c2]}', fontsize=title_size)
    ax_scat.tick_params(axis='both', which='major', labelsize=tick_size, labelbottom=True)

# 3. Format and Save
plt.tight_layout()

# output_dir = r'C:\Users\GeorgiaRg\Documents\Poros2024\Xact\Python_Output'
# os.makedirs(output_dir, exist_ok=True)
# output_path = os.path.join(output_dir, 'Ca_K_Zn_Fe_Timeseries_Scatter.png')
# plt.savefig(output_path, dpi=300, bbox_inches='tight')
# print(f"Plot saved successfully to: {output_path}")

plt.show()

## Create a PP with ALL the comparisons (timeseries and scatterplots)

In [ ]:
# Get all columns except the date column
data_cols = [col for col in df.columns if col.lower() != 'date']

# Generate all unique pairs of columns
column_pairs = list(itertools.combinations(data_cols, 2))

# 2. Initialize the PowerPoint Presentation
prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]

print(f"Total slides to generate: {len(column_pairs)}")

# 3. Loop through each pair to generate plots and slides
for col1, col2 in column_pairs:

    # Create a figure with 1 row and 2 columns
    # gridspec_kw makes the timeseries (left) twice as wide as the scatter plot (right)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5.5), gridspec_kw={'width_ratios': [2, 1]})

    # --- LEFT PLOT: TIMESERIES (LINES) ---
    is_col1_pm01 = 'pm0.1' in col1.lower()
    is_col2_pm01 = 'pm0.1' in col2.lower()

    # If exactly ONE of the columns is pm0.1, put it on the right axis
    if is_col1_pm01 != is_col2_pm01:
        primary_col = col2 if is_col1_pm01 else col1
        secondary_col = col1 if is_col1_pm01 else col2

        # Plot Primary Line
        ax1.plot(df['date'], df[primary_col], label=primary_col, color='tab:blue')
        ax1.set_ylabel(primary_col, color='tab:blue')
        ax1.tick_params(axis='y', labelcolor='tab:blue')

        # Plot Secondary Line on twin axis
        ax1_twin = ax1.twinx()
        ax1_twin.plot(df['date'], df[secondary_col], label=secondary_col, color='tab:orange')
        ax1_twin.set_ylabel(secondary_col, color='tab:orange')
        ax1_twin.tick_params(axis='y', labelcolor='tab:orange')

        # Combine legends from both axes
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax1_twin.get_legend_handles_labels()
        ax1.legend(lines + lines2, labels + labels2, loc='upper left')

    else:
        # If both or neither are pm0.1, plot them on the same axis
        ax1.plot(df['date'], df[col1], label=col1, color='tab:blue')
        ax1.plot(df['date'], df[col2], label=col2, color='tab:orange')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='upper left')

    ax1.set_xlabel('Date')
    ax1.set_title(f'Timeseries: {col1} and {col2}')
    fig.autofmt_xdate()

    # --- RIGHT PLOT: SCATTER WITH R2 ---
    scatter_data = df[[col1, col2]].dropna()

    if len(scatter_data) > 1:
        r = scatter_data[col1].corr(scatter_data[col2])
        r_squared = r**2

        ax2.scatter(scatter_data[col1], scatter_data[col2], alpha=0.6, color='tab:red')

        ax2.text(0.05, 0.95, f'$R^2 = {r_squared:.3f}$',
                 transform=ax2.transAxes,
                 fontsize=12, verticalalignment='top',
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'))
    else:
        ax2.text(0.5, 0.5, 'Insufficient overlapping data',
                 transform=ax2.transAxes, ha='center', va='center')

    ax2.set_xlabel(col1)
    ax2.set_ylabel(col2)
    ax2.set_title(f'Scatter: {col1} vs {col2}')

    plt.tight_layout()

    # --- SAVE TO MEMORY AND ADD TO PPTX ---
    image_stream = io.BytesIO()
    plt.savefig(image_stream, format='png', dpi=100)
    plt.close(fig)
    image_stream.seek(0)

    slide = prs.slides.add_slide(blank_slide_layout)
    slide.shapes.add_picture(image_stream, Inches(0.5), Inches(1), width=Inches(9))

# Join the path and the filename together safely
ppt_filepath = os.path.join(output_dir, 'Data_Analysis_Plots_Lines.pptx')

prs.save(ppt_filepath)
print(f"Presentation saved successfully to: {ppt_filepath}")